In [2]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_6978/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/19 16:32:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/19 16:32:53 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [13]:
import pandas as pd

# Sample data for HallEvents table
hall_events_data = {
    'hall_id': [1, 1, 1, 1, 2, 2, 3],
    'start_day': ['2023-01-12', '2023-01-14', '2023-01-18', '2023-01-13', 
                  '2022-12-09', '2022-12-13', '2022-12-01'],
    'end_day': ['2023-01-14', '2023-01-17', '2023-01-25', '2023-01-18',
                '2022-12-23', '2022-12-17', '2023-01-30']
}

# # Sample data for HallEvents table
# hall_events_data = {
#     'hall_id': [1, 1, 1, 2, 2, 3],
#     'start_day': ['2023-01-13', '2023-01-14', '2023-01-18', 
#                   '2022-12-09', '2022-12-13', '2022-12-01'],
#     'end_day': ['2023-01-14', '2023-01-17', '2023-01-25',
#                 '2022-12-23', '2022-12-17', '2023-01-30']
# }

# Create DataFrame for HallEvents
hall_events_df = pd.DataFrame(hall_events_data)

# Convert date columns to datetime type
hall_events_df['start_day'] = pd.to_datetime(hall_events_df['start_day'])
hall_events_df['end_day'] = pd.to_datetime(hall_events_df['end_day'])

# Display the DataFrame
print("HallEvents table:")
print(hall_events_df)


df_person = spark.createDataFrame(hall_events_df)
df_person.createOrReplaceTempView("HallEvents")

HallEvents table:
   hall_id  start_day    end_day
0        1 2023-01-12 2023-01-14
1        1 2023-01-14 2023-01-17
2        1 2023-01-18 2023-01-25
3        1 2023-01-13 2023-01-18
4        2 2022-12-09 2022-12-23
5        2 2022-12-13 2022-12-17
6        3 2022-12-01 2023-01-30


In [14]:
query = """
    select
        hall_id,
        min(start_day),
        max(end_day)
    from (
        select
            hall_id,
            sum(
                if(overlap_with_past, 0, 1)
            ) over (partition by hall_id order by start_day asc) as group_id,
            start_day,
            end_day,
            max_end_day_past,
            overlap_with_past
        from (
            select
                *,
                max(end_day) over(partition by hall_id order by start_day asc ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) as max_end_day_past,
                ifnull(start_day <= max(end_day) over(partition by hall_id order by start_day asc ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING), False) as overlap_with_past
            from
                HallEvents h
        )
    ) group by hall_id, group_id
"""

In [15]:
spark.sql(query).show()

+-------+-------------------+-------------------+
|hall_id|     min(start_day)|       max(end_day)|
+-------+-------------------+-------------------+
|      1|2023-01-12 00:00:00|2023-01-25 00:00:00|
|      2|2022-12-09 00:00:00|2022-12-23 00:00:00|
|      3|2022-12-01 00:00:00|2023-01-30 00:00:00|
+-------+-------------------+-------------------+

